# COGS 108 - Data Checkpoint

# Names

- Ant Man
- Hulk
- Iron Man
- Thor
- Wasp

<a id='research_question'></a>
# Research Question

*Fill in your research question here*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [1]:
## YOUR CODE HERE

# # upgrade pip
!{sys.executable} -m pip install --upgrade pip

#install external libraries
import sys
!{sys.executable} -m pip install nba_api # nba stats
!{sys.executable} -m pip install pandera # data validation 
# !{sys.executable} -m pip install pandera[io] # data validation + yaml output --error

# libraries
import pandas as pd
import numpy as np
import os

# team info
from nba_api.stats.static import teams

# team stats
from nba_api.stats.endpoints import teamyearbyyearstats as teamyears

import pandera as pa
from pandera import Check, Column, DataFrameSchema
from pathlib import Path

/bin/bash: {sys.executable}: command not found
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 122 kB 9.9 MB/s eta 0:00:01
     |████████████████████████████████| 13.2 MB 18.5 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 6.4 MB/s  eta 0:00:01


/opt/conda/lib/python3.9/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/opt/conda/lib/python3.9/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/opt/conda/lib/python3.9/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


# Data Cleaning

Describe your data cleaning steps here.

In [2]:
import time
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION
team_ids = [t['id'] for t in teams.get_teams()]
df = pd.DataFrame()

# slow but effective
for i in team_ids:
    df = pd.concat([df, teamyears.TeamYearByYearStats(i).get_data_frames()[0]])
    time.sleep(0.2) # any lower and it'll time out
print(df)

       TEAM_ID   TEAM_CITY   TEAM_NAME     YEAR  GP  WINS  LOSSES  WIN_PCT  \
0   1610612737  Tri-Cities  Blackhawks  1949-50  64    29      35    0.453   
1   1610612737  Tri-Cities  Blackhawks  1950-51  68    25      43    0.368   
2   1610612737   Milwaukee       Hawks  1951-52  66    17      49    0.258   
3   1610612737   Milwaukee       Hawks  1952-53  71    27      44    0.380   
4   1610612737   Milwaukee       Hawks  1953-54  72    21      51    0.292   
..         ...         ...         ...      ...  ..   ...     ...      ...   
28  1610612766   Charlotte     Hornets  2018-19  82    39      43    0.476   
29  1610612766   Charlotte     Hornets  2019-20  65    23      42    0.354   
30  1610612766   Charlotte     Hornets  2020-21  72    33      39    0.458   
31  1610612766   Charlotte     Hornets  2021-22  82    43      39    0.524   
32  1610612766   Charlotte     Hornets  2022-23   8     3       5    0.375   

    CONF_RANK  DIV_RANK  ...  OREB  DREB   REB   AST    PF  STL

## [Team Stats](https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/teamyearbyyearstats.md)

**gameplan for anish and baraa**

- data overview
    - look over the [nba_api website](https://github.com/swar/nba_api) to fully understand what we're working with
    - read in the data from jackson
- cleaning
    - get team stats (n = 30)
        - how? team id/name
    - potential columns needed: 
    - steps 
        1. **Validity**
            1. data type and range (numbers and dates) constraints 
            1. foreign key (FK) constraint (FK column can't have a value that doesn't exist in the referenced primary key )
            1. cross-field validation (e.g. ```FG3M``` $\leq$ ```FG3A```)
            1. **to make this go faster, the [pandera](https://pypi.org/project/pandera/) library can validate our data** 
                1. [Article 1](https://towardsdatascience.com/data-validation-for-pandas-b24613959364)
                1. [Article 2](https://towardsdatascience.com/validate-your-pandas-dataframe-with-pandera-2995910e564)
        1. **Inspect**
            1. summary statistics
            1. missing values
            1. unique values 
            1. distribution
            1. relationship (```FG3A``` &harr; ```FG3M```)
            1. visualize for outliers
            1. ```great_expectations``` library
        1. **Clean**
            1. irrelevant data (remove unneeded columns)
            1. drop duplicates
            1. type conversion (e.g. dates &rarr; ```df["col"] = to_datetime(df["col"])```)
            1. remove white space
            1. fix typos (**ewwww**)
            1. check for missing data that is labeled (e.g. ```0```, ```NA```, ```INF```)
            1. standardize (e.g. Strings &rarr; all upper/lower case, Season ```2022-2023``` &rarr; ```2022```)
            1. scale?? 
            1. missing values!!! 
                - 3 options: Drop if rare, impute (mean if data is not skewed, median if data is skewed), leave it as is/recode to a missing indicator)
                - *missing data doesn't equal unknown* per se (NBA website is relaible, so we shouldn't deal with this hopefully)
            1. create new columns &rarr; data transformation? 
        1. **Verify**
            1. Redo constraints check (see validation) w/ ```pandera``` library **assuming if we replaced the missing data**
        6. **Export Data** 
            1. One DF as a ```csv``` file

In [ ]:
df.sample(5)

In [ ]:
os.makedirs("data", exist_ok = True)
df.to_csv("data/nba_stats_raw.csv")

## Validation

1. data type and range (numbers and dates) constraints
1. foreign key (FK) constraint (FK column can't have a value that doesn't exist in the referenced primary key )
1. cross-field validation (e.g. ```FG3M``` $\leq$ ```FG3A```)
1. **to make this go faster, the [pandera](https://pandera.readthedocs.io/en/stable/index.html) library can validate our data** 
    1. [Article 1](https://towardsdatascience.com/data-validation-for-pandas-b24613959364)
    1. [Article 2](https://towardsdatascience.com/validate-your-pandas-dataframe-with-pandera-2995910e564)

In [ ]:
df = df[["TEAM_ID", "TEAM_CITY", "TEAM_NAME", "YEAR", "GP", "WINS", "LOSSES", "WIN_PCT",
         "PO_WINS", "PO_LOSSES", "NBA_FINALS_APPEARANCE",
   "FGM", "FGA", "FG_PCT", "FG3M", "FG3A", "FG3_PCT", "FTM", "FTA", "FT_PCT", "PTS", "PTS_RANK"]]
df.columns

In [ ]:
team_ids = [t["id"] for t in teams.get_teams()]
team_nickname = [t["nickname"] for t in teams.get_teams()]

In [ ]:
schema = pa.DataFrameSchema(
    {
        "TEAM_ID": Column(int, Check.isin(team_ids)),
        "TEAM_CITY": Column(str),
        "TEAM_NAME": Column(str, Check.isin(team_nickname)),
        "YEAR": Column(str),
        "GP": Column(int, Check.greater_than(0), Check.less_than(83)),
        "WINS": Column(int, Check.greater_than(-1), Check.less_than(83)),
        "LOSSES": Column(int, Check.greater_than(-1), Check.less_than(83)),
        "WIN_PCT": Column(float, Check.greater_than(-1), Check.less_than(1.0001)),
        "PO_WINS": Column(int, Check.greater_than(-1), Check.less_than(29)),
        "PO_LOSSES": Column(int, Check.greater_than(-1), Check.less_than(29)),
        "NBA_FINALS_APPEARANCE": Column(str),
        "FGM": Column(int, Check.greater_than(-1)),
        "FGA": Column(int, Check.greater_than(-1)),
        "FG_PCT": Column(float, Check.greater_than(-1), lambda x: x == (x["FGM"] / x["FGA"])),
        "FG3M": Column(int, Check.greater_than(-1)),
        "FG3A": Column(int, Check.greater_than(-1)),
        "FG3_PCT": Column(float, Check.greater_than(-1), lambda x: x == (x["FG3M"] / x["FG3A"])),
        "FTM": Column(int, Check.greater_than(-1)),
        "FTA": Column(int, Check.greater_than(-1)),
        "FT_PCT": Column(float, Check.greater_than(-1), lambda x: x == (x["FTM"] / x["FTA"])),
        "PTS": Column(int),
        "PTS_RANK": Column(int, Check.greater_than(0), Check.less_than(31)),   
    }
)

In [ ]:
schema_errors = pd.DataFrame()
df_errors = pd.DataFrame()

try:
    schema.validate(df, lazy=True)
except pa.errors.SchemaErrors as err:
    print("Schema errors and failure cases:")
    #print(err.failure_cases)
    schema_errors = err.failure_cases
    print("\nDataFrame object that failed validation:")
    #print(err.data)
    df_errors = err.data

In [ ]:
schema_errors

In [ ]:
schema_errors[schema_errors["failure_case"] == "Bullets"]

In [ ]:
df_errors

## Inspection

1. summary statistics
1. missing values 
1. unique values 
1. distribution
1. relationship (```FG3A``` &harr; ```FG3M```)
1. visualize for outliers
1. ```great_expectations``` library

In [ ]:
1 + 1

## Cleaning

1. irrelevant data (remove unneeded columns)
1. drop duplicates
1. type conversion (e.g. dates &rarr; ```df["col"] = to_datetime(df["col"])```)
1. remove white space
1. fix typos (**ewwww**)
1. check for missing data that is labeled (e.g. ```0```, ```NA```, ```INF```)
1. standardize (e.g. Strings &rarr; all upper/lower case, Season ```2022-2023``` &rarr; ```2022```)
1. scale?? 
1. missing values!!! 
    - 3 options: Drop if rare, impute (mean if data is not skewed, median if data is skewed), leave it as is/recode to a missing indicator)
    - *missing data doesn't equal unknown* per se (NBA website is relaible, so we shouldn't deal with this hopefully)
1. create new columns &rarr; data transformation? 

## Verification

1. Redo constraints check (see validation) w/ ```pandera``` library **assuming if we replaced the missing data**

In [ ]:
#

## Export Data

1. One DF as a ```csv``` file

In [ ]:
df.to_csv("data/nba_stats_cleaned.csv")